In [4]:
from dotenv import load_dotenv
from pyzotero import zotero
import os
from pprint import pprint, pformat

In [5]:
load_dotenv()
zot = zotero.Zotero(os.getenv("ZOTERO_ID"), "user", os.getenv("ZOTERO_API_KEY"), local=False)

In [6]:
zot.count_items()

4224

In [7]:
collections = zot.collections()
collection_dict = {}
for collection in collections:
    collection_dict[collection['data']['name']] = collection['data']['key']
pprint(collection_dict)

{'Can neural mass models accurately capture synaptic plasticity mechanisms - 13 Jul 2025': 'WKYIZNNF',
 'Cited in - Neuroplasticity Thread': '4ZH73WBS',
 'Cited in entropyThread - 01 Aug 2025': 'WPXG7R5M',
 'ComplexityEntropyChaos': 'ZVCGI7JK',
 'ConsensusCitation - 31 Jul 2025': 'KMSLSSL7',
 'EEG': 'IK26UWYI',
 'EEG_For_Meditation': '752GWEEQ',
 'FundamentalBooks': 'I6MYWKWQ',
 'How are neuroplasticity, learning and recovery modeleld in state of the art brain models like Neural Mass Models - 13 Jul 2025': '88AFQG3V',
 'How can mechanistic synaptic plasticity rules be computationally scaled and integrated into neural mass models to create biologically realistic, adaptive brain network simulations that capture learning, recovery, and pathological dynamics - 13 Jul 25': 'CL6BNXEM',
 'IIT': 'PBCQNZV2',
 'MasterThesis': 'VEG5XGN8',
 'Mystical Entropy': 'PH7L637T',
 'Neuroplasticity+NMM-review': 'QPEWATYL',
 'NotionSync': 'B6GWT9RN',
 'PCI-From-Resting-State-Reconstruction': '574RX5AA',
 'P

In [8]:
item_dict = {}
for item in zot.collection_items('9K7XL6ZV'):
    item_id = item.get('key')
    item_name = item.get('data', {}).get('title')
    item_dict[item_name] = item_id
pprint(item_dict)

{'A theoretically based index of consciousness independent of sensory processing and behavior': 'MQJ696B2',
 'Application of a neural complexity measure to multichannel EEG': 'XF9YTG4C',
 'Are we really unconscious in “unconscious” states? Common assumptions revisited': 'U59UVCUQ',
 'Attachment': 'RX4TQM7G',
 'Consciousness is supported by near-critical slow cortical electrodynamics': 'FMFSIDY8',
 'Criticality of resting-state EEG predicts perturbational complexity and level of consciousness during anesthesia': 'PTKEZXUM',
 'Criticality supports cross-frequency cortical-thalamic information transfer during conscious states': 'NZKUHVRF',
 'Editorial: Dissociations between neural activity and conscious state: a key to understanding consciousness': 'DTYLUE9L',
 'Explanation of near-death experiences: a systematic analysis of case reports and qualitative research': 'AIBUGQZZ',
 'Full Text': 'EJCNRRMT',
 'Full Text PDF': 'UNVKEPY3',
 'Human Consciousness: Where Is It From and What Is It for

In [9]:
pprint(zot.item('MQJ696B2'))

{'data': {'DOI': '10.1126/scitranslmed.3006294',
          'ISSN': '1946-6242',
          'abstractNote': 'One challenging aspect of the clinical assessment '
                          'of brain-injured, unresponsive patients is the lack '
                          'of an objective measure of consciousness that is '
                          "independent of the subject's ability to interact "
                          'with the external environment. Theoretical '
                          'considerations suggest that consciousness depends '
                          "on the brain's ability to support complex activity "
                          'patterns that are, at once, distributed among '
                          'interacting cortical areas (integrated) and '
                          'differentiated in space and time '
                          '(information-rich). We introduce and test a '
                          'theory-driven index of the level of consciousness '
           

In [14]:
def local_pdf_id_from_item_id(item_id: str) -> str:
    item = zot.item(item_id)
    links = item.get('links', {})
    attachment = links.get('attachment', {})
    if not attachment.get('attachmentType') == 'application/pdf':
        return ''
    href = attachment.get('href', '')
    local_id = href.split(os.sep)[-1]
    return local_id

def pdf_path_from_local_pdf_id(local_id: str) -> str:
    storage_path = os.path.join(os.getenv('LOCAL_ZOTERO_PATH'), 'storage', local_id)
    if not (files := os.listdir(storage_path)):
        return ''
    for file in files:
        if file.endswith('.pdf'):
            pdf_path = os.path.join(storage_path, file)
            return pdf_path
    return ''

def local_pdf_path_from_item_id(item_id: str) -> (str, str):
    """Find the local pdf based on the item_id"""
    local_id = local_pdf_id_from_item_id(item_id)
    return pdf_path_from_local_pdf_id(local_id)


In [15]:
local_pdf_path_from_item_id('MQJ696B2')

'/home/soenke/Zotero/storage/BQXMN6NY/Casali et al. - 2013 - A theoretically based index of consciousness indep.pdf'

TypeError: Zotero.items() takes 1 positional argument but 2 were given